In [3]:
import pandas as pd
import requests
from xml.etree import ElementTree

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df_SP500 = df[0].loc[:,['Symbol', 'Security','GICS Sector']]
df_SP500.tail()

,Symbol,Security,GICS Sector
500,YUM,Yum! Brands Inc,Consumer Discretionary
501,ZBRA,Zebra Technologies,Information Technology
502,ZBH,Zimmer Biomet,Health Care
503,ZION,Zions Bancorp,Financials
504,ZTS,Zoetis,Health Care


In [6]:
sentences = []
for i in range(len(df_SP500)):
    name = df_SP500['Security'][i]
    if '&' in name:
        name = ''.join(name.split('&'))
    print(name)
    x = requests.get('https://lookup.dbpedia.org/api/search/KeywordSearch?MaxHits=1&QueryString=' + name)
    tree = ElementTree.fromstring(x.content)
#     for elem in tree.iter(tag='Description'):
#         print(elem.text + '\n')
#         break
    sentences.append(tree[0][2].text)
df_SP500['text'] = sentences
df_SP500.tail()
df_SP500.to_csv('./S&P.csv')

3M Company
Abbott Laboratories
AbbVie Inc.
Abiomed
Accenture
Activision Blizzard
Adobe Inc.
Advanced Micro Devices
Advance Auto Parts
AES Corp
Aflac
Agilent Technologies
Air Products  Chemicals
Akamai Technologies
Alaska Air Group
Albemarle Corporation
Alexandria Real Estate Equities
Alexion Pharmaceuticals
Align Technology
Allegion
Alliant Energy
Allstate Corp
Alphabet Inc. (Class A)
Alphabet Inc. (Class C)
Altria Group Inc
Amazon.com Inc.
Amcor
Ameren Corp
American Airlines Group
American Electric Power
American Express
American International Group
American Tower Corp.
American Water Works
Ameriprise Financial
AmerisourceBergen
Ametek
Amgen Inc.
Amphenol Corp
Analog Devices, Inc.
ANSYS, Inc.
Anthem
Aon plc
A.O. Smith Corp
APA Corporation
Apple Inc.
Applied Materials Inc.
Aptiv PLC
Archer-Daniels-Midland Co
Arista Networks
Arthur J. Gallagher  Co.
Assurant
ATT Inc.
Atmos Energy
Autodesk Inc.
Automatic Data Processing
AutoZone Inc
AvalonBay Communities
Avery Dennison Corp
Baker Hughes 

W. R. Berkley Corporation
Westinghouse Air Brake Technologies Corp
Walmart
Walgreens Boots Alliance
The Walt Disney Company
Waste Management Inc.
Waters Corporation
WEC Energy Group
Wells Fargo
Welltower Inc.
West Pharmaceutical Services
Western Digital
Western Union Co
WestRock
Weyerhaeuser
Whirlpool Corp.
Williams Companies
Willis Towers Watson
Wynn Resorts Ltd
Xcel Energy Inc
Xerox
Xilinx
Xylem Inc.
Yum! Brands Inc
Zebra Technologies
Zimmer Biomet
Zions Bancorp
Zoetis


In [4]:
df_SP500 = pd.read_csv('./S&P.csv', index_col=0)
df_SP500.head()

,Symbol,Security,GICS Sector,text
0,MMM,3M Company,Industrials,"The 3M Company, formerly known as the Minnesot..."
1,ABT,Abbott Laboratories,Health Care,Abbott Laboratories is an American medical dev...
2,ABBV,AbbVie Inc.,Health Care,AbbVie is an American publicly traded biopharm...
3,ABMD,Abiomed,Health Care,Abiomed is a manufacturer of medical implant d...
4,ACN,Accenture,Information Technology,"Accenture plc, stylised as accenture, is an Ir..."


In [2]:
import pandas as pd
import re
import spacy
import neuralcoref
nlp = spacy.load('en_core_web_lg')
neuralcoref.add_to_pipe(nlp)


def get_entity_pairs(text, coref=True):
    # preprocess text
    text = re.sub(r'\n+', '.', text)  # replace multiple newlines with period
    text = re.sub(r'\[\d+\]', ' ', text)  # remove reference numbers
    text = nlp(text)
    if coref:
        text = nlp(text._.coref_resolved)  # resolve coreference clusters

    def refine_ent(ent, sent):
        unwanted_tokens = (
            'PRON',  # pronouns
            'PART',  # particle
            'DET',  # determiner
            'SCONJ',  # subordinating conjunction
            'PUNCT',  # punctuation
            'SYM',  # symbol
            'X',  # other
        )
        ent_type = ent.ent_type_  # get entity type
        if ent_type == '':
            ent_type = 'NOUN_CHUNK'
            ent = ' '.join(str(t.text) for t in
                           nlp(str(ent)) if t.pos_
                           not in unwanted_tokens and t.is_stop == False)
        elif ent_type in ('NOMINAL', 'CARDINAL', 'ORDINAL') and str(ent).find(' ') == -1:
            refined = ''
            for i in range(len(sent) - ent.i):
                if ent.nbor(i).pos_ not in ('VERB', 'PUNCT'):
                    refined += ' ' + str(ent.nbor(i))
                else:
                    ent = refined.strip()
                    break

        return ent, ent_type

    sentences = [sent.string.strip() for sent in text.sents]  # split text into sentences
    ent_pairs = []
    for sent in sentences:
        sent = nlp(sent)
        spans = list(sent.ents) + list(sent.noun_chunks)  # collect nodes
        spans = spacy.util.filter_spans(spans)
        with sent.retokenize() as retokenizer:
            [retokenizer.merge(span, attrs={'tag': span.root.tag,
                                            'dep': span.root.dep}) for span in spans]
        deps = [token.dep_ for token in sent]

        # limit our example to simple sentences with one subject and object
        if (deps.count('obj') + deps.count('dobj')) != 1\
                or (deps.count('subj') + deps.count('nsubj')) != 1:
            continue

        for token in sent:
            if token.dep_ not in ('obj', 'dobj'):  # identify object nodes
                continue
            subject = [w for w in token.head.lefts if w.dep_
                       in ('subj', 'nsubj')]  # identify subject nodes
            if subject:
                subject = subject[0]
                # identify relationship by root dependency
                relation = [w for w in token.ancestors if w.dep_ == 'ROOT']
                if relation:
                    relation = relation[0]
                    # add adposition or particle to relationship
                    if relation.nbor(1).pos_ in ('ADP', 'PART'):
                        relation = ' '.join((str(relation), str(relation.nbor(1))))
                else:
                    relation = 'unknown'

                subject, subject_type = refine_ent(subject, sent)
                token, object_type = refine_ent(token, sent)

                ent_pairs.append([str(subject), str(relation), str(token),
                                  str(subject_type), str(object_type)])

    ent_pairs = [sublist for sublist in ent_pairs
                          if not any(str(ent) == '' for ent in sublist)]
    pairs = pd.DataFrame(ent_pairs, columns=['subject', 'relation', 'object',
                                             'subject_type', 'object_type'])
    print('Entity pairs extracted:', str(len(ent_pairs)))

    return pairs

In [56]:
result = pd.DataFrame([], columns=['subject', 'relation', 'object',
                                             'subject_type', 'object_type'])
for i in range(len(df_SP500)):
    try:
        pairs = get_entity_pairs(df_SP500.loc[i,'text'])
        result = pd.concat([result, pairs])
    except:
        print(i)
    

Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 2
Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 3
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 2
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 2
Entity pairs extracted: 1
Entity pairs extracted: 2
Entity pairs extracted: 2
Entity pairs extracted: 2
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs

Entity pairs extracted: 2
319
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 2
Entity pairs extracted: 1
Entity pairs extracted: 2
Entity pairs extracted: 0
Entity pairs extracted: 2
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 2
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 0
Entity pairs extracted: 2
Entity pairs extracted: 1
344
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 1
Entity pairs extracted: 2
Entity pairs extracted: 2
Entity pairs extracted: 3
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 0
Entity pairs extracted: 1
Enti

In [61]:
result.to_csv('./S&PTriplet.csv', index=0)

In [60]:
result

,subject,relation,object,subject_type,object_type
0,The 3M Company,produces,"60,000 products",ORG,NOUN_CHUNK
0,Michael R. Minogue,joined,Abiomed,PERSON,PERSON
0,Fortune Global 500 company,had,"150,000 employees",NOUN_CHUNK,NOUN_CHUNK
1,accenture current clients,include,92 of the Fortune Global 100 and more than thr...,NOUN_CHUNK,CARDINAL
0,"Activision Blizzard, Inc.",includes,business units,ORG,NOUN_CHUNK
...,...,...,...,...,...
0,Xerox Holdings Corporation,purchased,Affiliated Computer Services,ORG,ORG
0,Xylem Inc.,does,business,ORG,NOUN_CHUNK
0,Zebra Technologies Corporation products,include,thermal barcode label,NOUN_CHUNK,NOUN_CHUNK
0,Keystone,acquired,57.5 percent interest,ORG,NOUN_CHUNK


In [17]:
df_SP500

,Symbol,Security,GICS Sector,text
0,MMM,3M Company,Industrials,"The 3M Company, formerly known as the Minnesot..."
1,ABT,Abbott Laboratories,Health Care,Abbott Laboratories is an American medical dev...
2,ABBV,AbbVie Inc.,Health Care,AbbVie is an American publicly traded biopharm...
3,ABMD,Abiomed,Health Care,Abiomed is a manufacturer of medical implant d...
4,ACN,Accenture,Information Technology,"Accenture plc, stylised as accenture, is an Ir..."
...,...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary,"Yum! Brands, Inc., or Yum! and formerly Tricon..."
501,ZBRA,Zebra Technologies,Information Technology,Zebra Technologies Corporation is an American ...
502,ZBH,Zimmer Biomet,Health Care,Zimmer Biomet is a publicly traded medical dev...
503,ZION,Zions Bancorp,Financials,Zions Bancorporation is a bank holding company...
